# Setting up

In [1]:
# Common
import os
import cv2 as cv
import numpy as np
from IPython.display import clear_output as cls
import re

# Data 
from tqdm import tqdm
from glob import glob

# Data Visuaalization
import plotly.express as px
import matplotlib.pyplot as plt

# Model
from tensorflow.keras.models import load_model
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from imageio import imread
from skimage.transform import resize

# Predefined variables
image_size =160

In [2]:
# Specify the root directory path
root_path = 'D:/Face recognition/VN-celeb/'

# Collect all the person names
dir_names = os.listdir(root_path)
person_names = [name for name in dir_names]
n_individuals = len(person_names)

print(f"Total number of individuals: {n_individuals}/n")

Total number of individuals: 1020/n


In [3]:
cascade_path = '../model/cv2/haarcascade_frontalface_alt2.xml'

In [4]:
model = load_model('../model/facenet-keras/facenet_keras.h5')

## Ultility functions

In [5]:
def get_train_test_split(rootpath, dir_names, train_size=0.8):
    train_filepaths, test_filepaths = [], []
    dir_names = os.listdir(rootpath)
    for name in tqdm(dir_names, desc='Train test split', total=len(dir_names)):
        count = 0
        file_names = os.listdir(rootpath + name)
        # print(file_names)
        train_temp, test_temp = [], []
        for fn in file_names:
            file_path = rootpath + name + '/' + fn
            if count < len(file_names)*train_size:
                train_temp.append(file_path)
            else:
                test_filepaths.append(file_path)
            count += 1
        train_filepaths.append(train_temp)
        # test_filepaths.append(test_temp)
    return train_filepaths, test_filepaths

In [6]:
def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

In [38]:
def load_and_align_images(filepaths, margin):
    cascade = cv.CascadeClassifier(cascade_path)
    
    aligned_images = []
    for filepath in filepaths:
        img = imread(filepath)

        faces = cascade.detectMultiScale(img,
                                         scaleFactor=1.1,
                                         minNeighbors=3)
        if len(faces) > 0:
            (x, y, w, h) = faces[0]
            x1 = max(0, x - margin//2)
            y1 = max(0, y - margin//2)
            x2 = min(img.shape[1], x + w + margin//2)
            y2 = min(img.shape[0], y + h + margin//2)
            # cropped = img[y-margin//2:y+h+margin//2,
            #               x-margin//2:x+w+margin//2, :]
            cropped = img[y1:y2, x1:x2, :]
            aligned = resize(cropped, (image_size, image_size), mode='reflect')
            aligned_images.append(aligned)
    print("Dimensions of aligned images:", [img.shape for img in aligned_images])       
    return np.array(aligned_images)

In [39]:
def calc_embs(filepaths, margin=10, batch_size=1):
    aligned_images = prewhiten(load_and_align_images(filepaths, margin))
    pd = []
    for start in range(0, len(aligned_images), batch_size):
        pd.append(model.predict_on_batch(aligned_images[start:start+batch_size]))
    embs = l2_normalize(np.concatenate(pd))

    return embs

In [40]:
def train(dir_basepath, names, max_num_img=10):
    labels = []
    embs = []
    for i, name in tqdm(enumerate(names), total=len(names)):
        # dirpath = os.path.abspath(dir_basepath + name)
        # filepaths = [os.path.join(dirpath, f) for f in os.listdir(dirpath)][:max_num_img]
        filepaths = dir_basepath[i][:max_num_img]
        # print(filepaths)
        embs_ = calc_embs(filepaths)    
        labels.extend([name] * len(embs_))
        embs.append(embs_)
        
    embs = np.concatenate(embs)
    le = LabelEncoder().fit(labels)
    y = le.transform(labels)
    clf = SVC(kernel='linear', probability=True).fit(embs, y)
    return le, clf

In [41]:
def infer(le, clf, filepaths):
    embs = calc_embs(filepaths)
    pred = le.inverse_transform(clf.predict(embs))
    return pred

# Train test split

In [42]:
trainpaths, testpaths = get_train_test_split(root_path, dir_names)
len(trainpaths), len(testpaths)

Train test split: 100%|██████████| 1020/1020 [00:00<00:00, 8233.40it/s]


(1020, 4203)

In [43]:
test_labels = []
for p in tqdm(testpaths, total=len(testpaths), desc='Creating test data labels'):
    label = re.findall(r'VN-celeb/(\d+)/', p)[0]
    test_labels.append(label)


Creating test data labels: 100%|██████████| 4203/4203 [00:00<00:00, 370225.55it/s]


# Run model

In [44]:
le, clf = train(trainpaths, dir_names)

  0%|          | 0/1020 [00:00<?, ?it/s]d:\Face recognition\facenet\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  


Dimensions of aligned images: [(160, 160, 3), (160, 160, 3), (160, 160, 3), (160, 160, 3), (160, 160, 3), (160, 160, 3), (160, 160, 3), (160, 160, 3)]


  0%|          | 1/1020 [00:00<15:57,  1.06it/s]

Dimensions of aligned images: [(160, 160, 3), (160, 160, 3), (160, 160, 3), (160, 160, 3), (160, 160, 3), (160, 160, 3), (160, 160, 3)]


  0%|          | 2/1020 [00:01<13:37,  1.25it/s]

...

 10%|▉         | 97/1020 [01:13<12:05,  1.27it/s]

Dimensions of aligned images: [(160, 160, 3), (160, 160, 3), (160, 160, 3), (160, 160, 3), (160, 160, 3), (160, 160, 3)]


 10%|▉         | 98/1020 [01:14<11:38,  1.32it/s]

Dimensions of aligned images: []


ValueError: Dimension should be 3 or 4

In [18]:
cascade = cv.CascadeClassifier(cascade_path)

In [57]:
img = imread(trainpaths[98][3])
# img

d:\Face recognition\facenet\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  """Entry point for launching an IPython kernel.


In [54]:
faces = cascade.detectMultiScale(img,
                                         scaleFactor=1.1,
                                         minNeighbors=3)

In [56]:
faces

()

In [48]:
x

NameError: name 'x' is not defined

In [49]:
(x, y, w, h) = faces[0]

In [50]:
margin=10
cropped = img[y-margin//2:y+h+margin//2,
                      x-margin//2:x+w+margin//2, :]

In [51]:
cropped

array([[[ 49,  44,  27],
        [ 50,  45,  30],
        [ 51,  45,  33],
        ...,
        [223,  66,  73],
        [202,  36,  47],
        [213,  47,  59]],

       [[ 56,  48,  30],
        [ 57,  49,  34],
        [ 62,  53,  40],
        ...,
        [214,  55,  64],
        [200,  34,  46],
        [225,  58,  72]],

       [[ 68,  56,  38],
        [ 67,  53,  37],
        [ 65,  52,  37],
        ...,
        [208,  46,  53],
        [203,  35,  44],
        [225,  55,  68]],

       ...,

       [[ 87,  35,  23],
        [ 92,  41,  29],
        [ 95,  39,  28],
        ...,
        [252,  68,  60],
        [246,  70,  67],
        [241,  85,  88]],

       [[ 94,  35,  22],
        [ 93,  41,  27],
        [ 96,  44,  31],
        ...,
        [252,  64,  62],
        [255,  70,  72],
        [250,  77,  80]],

       [[ 97,  34,  21],
        [ 90,  38,  24],
        [ 91,  39,  28],
        ...,
        [249,  61,  58],
        [254,  69,  69],
        [251,  77,  79]]

In [ ]:
aligned = resize(cropped, (image_size, image_size), mode='reflect')

ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
if len(faces) > 0:
    (x, y, w, h) = faces[0]
    
    # Adjust cropping region to ensure it stays within image boundaries
    x1 = max(0, x - margin//2)
    y1 = max(0, y - margin//2)
    x2 = min(img.shape[1], x + w + margin//2)
    y2 = min(img.shape[0], y + h + margin//2)
    
    # Perform cropping with adjusted coordinates
    cropped = img[y1:y2, x1:x2, :]
    
    # Check if cropped array is empty
    if cropped.size == 0:
        print("Cropping region is outside image boundaries")
    else:
        # Resize the cropped image to the desired size
        aligned = resize(cropped, (image_size, image_size), mode='reflect')
else:
    print("No faces detected")